In [ ]:
!pip install datasets
!pip install peft
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 25.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-non

In [ ]:
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
import random
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
class LateFusionEnsembler(nn.Module):
  def __init__(self,model1,model2,tokenizer,modelSize=32000,device="cuda"):
    super().__init__()
    self.model1 = model1.to(device)
    self.model2 = model2.to(device)
    self.tokenizer = tokenizer
    self.modelSize = modelSize
    self.device = device
    self.linear1 = nn.Linear(self.modelSize*2,4)
    # self.relu = nn.ReLU()
    # self.linear2 = nn.Linear(8000,4)

  def forward(self,inputIndices):
    y1 = self.model1(inputIndices).logits
    y2 = self.model2(inputIndices).logits

    n,h,w = y1.shape

    y1 = y1[:,h-1,:]
    y2 = y2[:,h-1,:]

    y= torch.cat((y1,y2),dim=1)

    y = self.linear1(y)
    # y = self.relu(y)
    # y = self.linear2(y)

    return y

In [ ]:
from datasets import load_dataset, load_from_disk, concatenate_datasets, Dataset,DatasetDict
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    MistralForSequenceClassification,
    PretrainedConfig,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_location = "/content/drive/MyDrive/685 Final Project/Datasets/medmcqa-prompts"

In [ ]:
train_dataset = load_from_disk(f"{dataset_location}/train_prompts_micro.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_micro.hf")
eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_micro.hf")

# train_dataset = load_from_disk(f"{dataset_location}/train_prompts_mini.hf")
# test_dataset = load_from_disk(f"{dataset_location}/test_prompts_mini.hf")
# eval_dataset = load_from_disk(f"{dataset_location}/eval_prompts_mini.hf")

In [ ]:
train_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot'],
    num_rows: 2000
})

In [ ]:
eval_dataset

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name', 'prompt', 'label_one_hot'],
    num_rows: 500
})

In [ ]:
# Load pre-trained models
from unsloth import FastLanguageModel

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model_location = "/content/drive/MyDrive/685 Final Project/Models"

model1, tokenizer = FastLanguageModel.from_pretrained(model_location + "/unsloth_domain1",
                                                     max_seq_length=max_seq_length,
                                                     dtype=dtype,
                                                     load_in_4bit=load_in_4bit)

model2, tokenizer = FastLanguageModel.from_pretrained(model_location + "/ai2_arc_instruction_tuned_mistral_7b",
                                                     max_seq_length=max_seq_length,
                                                     dtype=dtype,
                                                     load_in_4bit=load_in_4bit)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [ ]:
from torch.utils.data import DataLoader, Dataset

class MCQDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)  # Changed to float for one-hot encoding
        return item

    def __len__(self):
        return len(self.labels)

# Function to encode the data
def encode_data(tokenizer, prompts):
    # encodings = tokenizer(prompts, truncation=True, padding=True, max_length=2048)
    encodings = tokenizer(prompts, truncation=True, padding=True)
    return encodings

# Prepare the data for tokenization
prompts = [item['prompt'] for item in train_dataset]
labels = [item['label_one_hot'] for item in train_dataset]  # one-hot encoded labels

# Tokenize data
encodings = encode_data(tokenizer, prompts)

# Create dataset
train_set = MCQDataset(encodings, labels)

# DataLoader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)


prompts = [item['prompt'] for item in eval_dataset]
labels = [item['label_one_hot'] for item in eval_dataset]  # one-hot encoded labels

# Tokenize data
encodings = encode_data(tokenizer, prompts)

# Create dataset
eval_set = MCQDataset(encodings, labels)

# DataLoader
val_loader = DataLoader(eval_set, batch_size=32, shuffle=True)

In [ ]:
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from torch.nn.functional import softmax
def train_and_validate(model, train_loader, val_loader, epochs=3):

    saved_model_location = "/content/drive/MyDrive/685 Final Project/Models"

    scaler = GradScaler()
    device = torch.device("cuda")
    model = model.to(device)  # Ensures model and all submodules are float32
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # for epoch in tqdm(range(epochs)):

    for epoch in range(epochs):
        total_train_loss = 0
        total_train_correct = 0
        train_samples = 0
        # correct=list()
        model.train()
        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [TRAIN]", unit="batch")
        for i, batch in enumerate(train_pbar):
            input_ids, labels = batch['input_ids'].to(device), batch['labels'].to(device)
            train_samples += labels.size(0)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(input_ids).float()
                loss = criterion(output, labels.float())
                predictions = torch.argmax(softmax(output,dim=1), dim=1)
                labels_indices = torch.argmax(labels, dim=1)

                train_correct = (predictions == labels_indices).sum().item()
                total_train_correct += train_correct
                # print("\nTotal Correct : ", train_correct)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_train_loss += loss.item()

            train_pbar.set_postfix(loss=loss.item(), temp_acc=100 * total_train_correct / train_samples)


            if i % 1000 == 0:
                print(i, loss.item())
                print(f"Temp accuracy: ", total_train_correct / train_samples * 100)

            # Releasing the memory
            del input_ids, labels, output, loss, predictions, labels_indices


        model_save_path = f"{saved_model_location}/LateFusion.pth"
        torch.save(model.state_dict(), model_save_path)
        print("model Saved at", model_save_path)


        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy = total_train_correct / train_samples * 100
        print(f"Training Accuracy: ", train_accuracy)
        print(f"Epoch {epoch+1}, Loss: {avg_train_loss}")

        model.eval()
        total_val_loss, val_samples, total_val_correct = 0, 0, 0
        with torch.no_grad():
            for i, batch in enumerate(val_loader):
                input_ids, labels = batch['input_ids'].to(device), batch['labels'].to(device)
                with torch.cuda.amp.autocast():
                    outputs = model(input_ids).float()
                    val_loss = criterion(outputs, labels.float())
                    predictions = torch.argmax(softmax(outputs,dim=1), dim=1)
                    labels_indices = torch.argmax(labels, dim=1)
                    total_val_correct += (predictions == labels_indices).sum().item()

                total_val_loss += val_loss.item()
                val_samples += labels.size(0)

        avg_val_loss = total_val_loss / len(val_loader)
        val_accuracy = total_val_correct / val_samples * 100
        print(f"Validation Accuracy: ", val_accuracy)
        print(f"Epoch {epoch+1} - Validation Loss: {avg_val_loss:.4f}")



In [ ]:
torch.cuda.empty_cache()

In [ ]:
lf = LateFusionEnsembler(model1,model2,tokenizer)

In [ ]:
train_and_validate(lf,train_loader,val_loader,epochs=1)

Epoch 1 [TRAIN]:   0%|          | 0/63 [00:00<?, ?batch/s]


Total Correct :  9


Epoch 1 [TRAIN]:   2%|▏         | 1/63 [00:06<06:48,  6.58s/batch, loss=1.52, temp_acc=28.1]

0 1.5157479047775269
Temp accuracy:  28.125

Total Correct :  5


Epoch 1 [TRAIN]:   3%|▎         | 2/63 [00:11<05:41,  5.60s/batch, loss=1.77, temp_acc=21.9]

...


Total Correct :  8


Epoch 1 [TRAIN]:  98%|█████████▊| 62/63 [05:07<00:04,  4.93s/batch, loss=1.66, temp_acc=25.8]


Total Correct :  3


Epoch 1 [TRAIN]: 100%|██████████| 63/63 [05:10<00:00,  4.93s/batch, loss=2.55, temp_acc=25.8]


model Saved at /content/drive/MyDrive/685 Final Project/Models/LateFusion0.pth
Training Accuracy:  25.75
Epoch 1, Loss: 3.063587164121961
Validation Accuracy:  26.400000000000002
Epoch 1 - Validation Loss: 2.0176


In [ ]:
model_path = '/content/drive/MyDrive/685 Final Project/Models/LateFusion0.pth'
lf.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
train_and_validate(lf,train_loader,val_loader,epochs=2)

Epoch 1 [TRAIN]:   2%|▏         | 1/63 [00:04<05:07,  4.95s/batch, loss=2.09, temp_acc=28.1]

0 2.091259002685547
Temp accuracy:  28.125


Epoch 1 [TRAIN]: 100%|██████████| 63/63 [05:08<00:00,  4.90s/batch, loss=1.43, temp_acc=26.3]


model Saved at /content/drive/MyDrive/685 Final Project/Models/LateFusion0.pth
Training Accuracy:  26.3
Epoch 1, Loss: 2.6051125829181974
Validation Accuracy:  32.800000000000004
Epoch 1 - Validation Loss: 1.5131


Epoch 2 [TRAIN]:   2%|▏         | 1/63 [00:04<05:06,  4.94s/batch, loss=1.71, temp_acc=18.8]

0 1.7141356468200684
Temp accuracy:  18.75


Epoch 2 [TRAIN]: 100%|██████████| 63/63 [05:08<00:00,  4.90s/batch, loss=1.72, temp_acc=28.2]


model Saved at /content/drive/MyDrive/685 Final Project/Models/LateFusion1.pth
Training Accuracy:  28.199999999999996
Epoch 2, Loss: 1.670744844845363
Validation Accuracy:  26.6
Epoch 2 - Validation Loss: 1.9734


In [ ]:
model_path = '/content/drive/MyDrive/685 Final Project/Models/LateFusion1.pth'
lf.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
train_and_validate(lf,train_loader,val_loader,epochs=2)

Epoch 1 [TRAIN]:   2%|▏         | 1/63 [00:06<06:24,  6.21s/batch, loss=1.48, temp_acc=43.8]

0 1.4807219505310059
Temp accuracy:  43.75


Epoch 1 [TRAIN]: 100%|██████████| 63/63 [05:10<00:00,  4.92s/batch, loss=1.39, temp_acc=26.2]


model Saved at /content/drive/MyDrive/685 Final Project/Models/LateFusion.pth
Training Accuracy:  26.25
Epoch 1, Loss: 2.369175250568087
Validation Accuracy:  21.2
Epoch 1 - Validation Loss: 1.3888


Epoch 2 [TRAIN]:   2%|▏         | 1/63 [00:04<05:06,  4.94s/batch, loss=1.47, temp_acc=21.9]

0 1.4694397449493408
Temp accuracy:  21.875


Epoch 2 [TRAIN]:   2%|▏         | 1/63 [00:09<10:00,  9.69s/batch, loss=1.47, temp_acc=21.9]


KeyboardInterrupt: 